In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append("../")


In [2]:
import numpy as np
from iotools import simulate

nsamples = 200
ngenes   = 100
nsnps    = 1000

expr = np.random.normal(0, 1, ngenes*nsamples).reshape(ngenes, nsamples)
snpinfo, gtnorm, gtcent = simulate.permuted_dosage(expr, nsnp = nsnps, fmin = 0.1, fmax = 0.9)


In [3]:
# Get genotype variance as in C
import numpy as np
import ctypes
import os

def gt_var(geno):
    clib = np.ctypeslib.load_library('../lib/reverse_regression.so', ".")
    cgt_var = clib.genotype_variance
    cgt_var.restype = ctypes.c_bool
    cgt_var.argtypes = [np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        ctypes.c_int,
                        ctypes.c_int,
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        ctypes.c_int                        
                       ]
    x = geno.reshape(-1,)
    nsnps = geno.shape[0]
    nsamples = geno.shape[1]
    SX2 = np.zeros(nsnps)
    success = cgt_var(x, nsnps, nsamples, SX2, 2)
    return SX2

In [4]:
sigma_x2 = gt_var(gtnorm)

In [5]:
U, s, VT = np.linalg.svd(expr.T, full_matrices=False)
print("U:", U.shape)
print("s:", s.shape)
print("VT:", VT.shape)

U: (200, 100)
s: (100,)
VT: (100, 100)


In [6]:
sigmabeta = 0.01
sigma_x2 = np.var(gtnorm,axis=1) # ** 2
sb2 = sigmabeta ** 2 # np.repeat ???
k = min(nsamples, ngenes)
I = np.identity(k)
S = I*s

In [7]:
# snpi = 0
# innerL = I*s*s + (sigma_x2[snpi]/sb2)*I
# innerL_inv = np.linalg.inv(innerL)
# V = np.transpose(VT)

# left_mat = np.matmul(V, innerL_inv)
# print("left_mat:", left_mat.shape)
# UT = np.transpose(U)

# StUt = np.matmul(S, UT)
# print("StUt:",StUt.shape) 

# A = np.matmul(left_mat, StUt)
# # B = np.matmul(A, X)
# print("A:",A.shape)

# Bi = np.matmul(A, gtnorm[1,:][np.newaxis].T)
# print(Bi[:10])

In [8]:
# innerL = (I*s*s + (sigma_x2/sb2)*I)
# innerL_inv = np.linalg.inv(innerL)
# test = np.matmul(innerL_inv, S)
# test
snpi = 1
innerLinv_ST = (s / (s*s + (sigma_x2[snpi]/sb2))) * I
print("innerLinv_ST:", innerLinv_ST.shape)
UT = np.transpose(U)
V = np.transpose(VT)

innerLinv_STUT = np.matmul(innerLinv_ST, UT)
print("innerLinv_STUT:", innerLinv_STUT.shape)  # if G < N, then UT is k x N and then innerL_STUT is k x N
A2 = np.matmul(V, innerLinv_STUT)
print("A:", A2.shape)

Bi = np.matmul(A2, gtnorm[snpi,:][np.newaxis].T)
print(Bi[:10])

innerLinv_ST: (100, 100)
innerLinv_STUT: (100, 200)
A: (100, 200)
[[ 5.73908827e-05]
 [-6.98346094e-04]
 [ 4.19881711e-04]
 [ 2.18642433e-03]
 [-7.65616412e-04]
 [-3.77948842e-04]
 [ 1.66053421e-03]
 [ 4.99618536e-04]
 [ 3.30592400e-04]
 [ 8.14125937e-04]]


In [9]:
import numpy as np
import ctypes
import os

def crevreg(geno, expr, sb2):
#     _path = os.path.dirname(".")
    clib = np.ctypeslib.load_library('../lib/reverse_regression.so', ".")
    cbetas = clib.betas
    cbetas.restype = ctypes.c_bool
    cbetas.argtypes = [np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        ctypes.c_int,
                        ctypes.c_int,
                        ctypes.c_int,
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED')
                       ]

    # print("expr shape is: genes x nsamples", expr.shape)
    # print("geno shape is: snps x nsamples", geno.shape)
    x = geno.reshape(-1,)
    y = expr.reshape(-1,)
    nsnps = geno.shape[0]
    nsamples = geno.shape[1]
    ngenes = expr.shape[0]
    B = np.zeros(ngenes * nsnps)
#     preB = np.zeros(ngenes * nsamples)
    success = cbetas(x, y, sb2, ngenes, nsnps, nsamples, B)
    return B


In [10]:
# print(s)
# print(s*s)
# print(sigma_x2[snpi:snpi+3])
# print(sb2)
# print(innerLinv_STUT[0:5,0:5])
# print(innerLinv_ST[0:5,0:5])
# print(UT[0:5,0:5])
# print(A2.shape)
# print(A2)

# print("Bi: ", Bi.shape)
# print(Bi.reshape(-1,)[:100])

In [11]:
sb2 = np.repeat(sigmabeta ** 2, gtnorm.shape[0])
B = crevreg(gtnorm, expr, sb2)

In [12]:
B[100:200]

array([ 5.73908827e-05, -6.98346094e-04,  4.19881711e-04,  2.18642433e-03,
       -7.65616412e-04, -3.77948842e-04,  1.66053421e-03,  4.99618536e-04,
        3.30592400e-04,  8.14125937e-04, -1.15058851e-03,  2.02365330e-03,
        2.32716688e-04, -1.07305879e-03, -1.90264285e-03,  1.26412002e-03,
       -1.43363922e-03, -1.42363302e-03,  2.42167521e-03,  1.46269643e-03,
        2.56912897e-04,  3.49421711e-05,  2.62220307e-03, -9.16627014e-04,
        2.31850133e-04,  4.79853002e-04,  4.22518409e-05,  3.50517590e-04,
        1.28339189e-04, -1.66701842e-03, -3.05569433e-04,  2.63456827e-03,
       -1.04498111e-03, -1.85787535e-03,  2.57604260e-03, -9.61487502e-05,
        2.29607229e-03, -5.69625496e-04,  5.14996067e-04,  2.90104581e-03,
        1.63148226e-03, -3.52476125e-03, -5.92813448e-04, -3.45417373e-04,
        9.05368483e-04, -3.03891228e-03, -2.67387809e-03, -1.74036708e-03,
        2.28340981e-04, -8.22848792e-04, -4.27462606e-03, -1.30157762e-04,
       -4.91782599e-04, -

In [13]:
Bi.reshape(-1,)

array([ 5.73908827e-05, -6.98346094e-04,  4.19881711e-04,  2.18642433e-03,
       -7.65616412e-04, -3.77948842e-04,  1.66053421e-03,  4.99618536e-04,
        3.30592400e-04,  8.14125937e-04, -1.15058851e-03,  2.02365330e-03,
        2.32716688e-04, -1.07305879e-03, -1.90264285e-03,  1.26412002e-03,
       -1.43363922e-03, -1.42363302e-03,  2.42167521e-03,  1.46269643e-03,
        2.56912897e-04,  3.49421711e-05,  2.62220307e-03, -9.16627014e-04,
        2.31850133e-04,  4.79853002e-04,  4.22518409e-05,  3.50517590e-04,
        1.28339189e-04, -1.66701842e-03, -3.05569433e-04,  2.63456827e-03,
       -1.04498111e-03, -1.85787535e-03,  2.57604260e-03, -9.61487502e-05,
        2.29607229e-03, -5.69625496e-04,  5.14996067e-04,  2.90104581e-03,
        1.63148226e-03, -3.52476125e-03, -5.92813448e-04, -3.45417373e-04,
        9.05368483e-04, -3.03891228e-03, -2.67387809e-03, -1.74036708e-03,
        2.28340981e-04, -8.22848792e-04, -4.27462606e-03, -1.30157762e-04,
       -4.91782599e-04, -

In [14]:
newB = B.reshape(nsnps, ngenes)

In [15]:
newB[1]

array([ 5.73908827e-05, -6.98346094e-04,  4.19881711e-04,  2.18642433e-03,
       -7.65616412e-04, -3.77948842e-04,  1.66053421e-03,  4.99618536e-04,
        3.30592400e-04,  8.14125937e-04, -1.15058851e-03,  2.02365330e-03,
        2.32716688e-04, -1.07305879e-03, -1.90264285e-03,  1.26412002e-03,
       -1.43363922e-03, -1.42363302e-03,  2.42167521e-03,  1.46269643e-03,
        2.56912897e-04,  3.49421711e-05,  2.62220307e-03, -9.16627014e-04,
        2.31850133e-04,  4.79853002e-04,  4.22518409e-05,  3.50517590e-04,
        1.28339189e-04, -1.66701842e-03, -3.05569433e-04,  2.63456827e-03,
       -1.04498111e-03, -1.85787535e-03,  2.57604260e-03, -9.61487502e-05,
        2.29607229e-03, -5.69625496e-04,  5.14996067e-04,  2.90104581e-03,
        1.63148226e-03, -3.52476125e-03, -5.92813448e-04, -3.45417373e-04,
        9.05368483e-04, -3.03891228e-03, -2.67387809e-03, -1.74036708e-03,
        2.28340981e-04, -8.22848792e-04, -4.27462606e-03, -1.30157762e-04,
       -4.91782599e-04, -